In [1]:
import cv2
import numpy as np
import heapq

In [2]:
# Open a video file or capture device
video_path = 'data/Data/kaniula1.avi'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Read the first frame from the video
ret, frame = cap.read()

# Display the first frame
cv2.imshow('First Frame', frame)
cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()

# Release the video capture object
cap.release()


In [17]:
image = frame
mask_height, mask_width = image.shape[:2]
mask = np.zeros((mask_height, mask_width), dtype=np.uint8)

mask_vertices = np.array([[mask_width*0.7, mask_height*0.3],
                             [mask_width*0.7, mask_height*0.7],
                             [mask_width*1.1, mask_height*3],
                             [mask_width*1.1, 0]], dtype=np.int32)
mask_vertices = mask_vertices.reshape((-1, 1, 2))

cv2.fillPoly(mask, [mask_vertices], 255)

result = cv2.bitwise_and(image, image, mask=mask)

cv2.imwrite("result_image.jpg", result)

cv2.imshow("Original Image", image)
cv2.imshow("Result Image", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.GaussianBlur(gray, (15, 15), 0)
    
    edges = cv2.Canny(blurred, 5, 10)
    
    return edges

In [19]:
def dijkstra_contour_detection(edges, start_coord, end_coord, mask):
    queue = []
    heapq.heappush(queue, (0, start_coord))

    while queue:
        distance, current = heapq.heappop(queue)
        x, y = current

        if current == end_coord:
            break

        neighbors = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]

        for neighbor in neighbors:
            nx, ny = neighbor
            if 0 <= nx < edges.shape[1] and 0 <= ny < edges.shape[0] and mask[ny, nx]:
                new_distance = distance + 1
                if new_distance < edges[ny, nx]:
                    edges[ny, nx] = new_distance
                    heapq.heappush(queue, (new_distance, (nx, ny)))

    return edges

In [22]:
def detect_guide_sheath_contours(frame):
    edges = preprocess_frame(frame)
    edges = cv2.bitwise_and(edges, np.ones_like(frame[:, :, 0], dtype=np.uint8) , mask=mask)
    
    contour = dijkstra_contour_detection(edges.copy(), (0, frame.shape[0] // 2), (frame.shape[1] - 1, frame.shape[0] // 2), np.ones_like(frame[:, :, 0], dtype=np.uint8))


    cv2.imshow('Original Frame', frame)
    cv2.imshow('Contour', contour.astype(np.uint8) * 255)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return contour.astype(np.uint8) * 255


In [23]:
image_contours = detect_guide_sheath_contours(frame)

In [12]:
cv2.imshow("im", image_contours)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
# Load your image
binary_image = image_contours

# Set parameters for adaptive thresholding
BOX = 11
OFFSET = 2

# Fill Holes within Blobs using morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
filled_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)


roi_mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

# Remove Blobs Outside the Mask
result_image = cv2.bitwise_and(filled_image, roi_mask)

# Display the result
cv2.imshow('Result Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()